<a href="https://colab.research.google.com/github/PATELOM925/Automatic-Paper-Checker/blob/main/Workling_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt-get install tesseract-ocr
!pip install pytesseract
!pip install pytesseract opencv-python pillow spacy
!pip install spacy
!python -m spacy download zh_core_web_lg



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
2023-10-13 04:32:20.584797: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.0/603.0 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_lg')


In [4]:
from google.colab import files
import cv2
import pytesseract
import spacy
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import numpy as np

# Use the files.upload() method to upload the student's and teacher's images
uploaded_student = files.upload()
uploaded_teacher = files.upload()

# Load spaCy model (you can choose a different language model if needed)
nlp = spacy.load("zh_core_web_lg")

# Initialize NLTK sentiment analyzer
nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()

# Data Preprocessing: Image Preprocessing
def preprocess_image(image_path):
    # Reading image
    image = cv2.imread(image_path, 0)  # Read as grayscale

    # Adaptive thresholding
    binary_image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtering contours
    min_contour_area = 100
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

    # Drawing contours on the original image
    segmented_image = cv2.drawContours(cv2.cvtColor(image, cv2.COLOR_GRAY2BGR), filtered_contours, -1, (0, 255, 0), 2)

    return segmented_image

# Function to perform OCR on the input image
def perform_ocr(image_path):
    # Data Preprocessing: Image Preprocessing
    segmented_image = preprocess_image(image_path)

    # OCR to extract text
    recognized_text = pytesseract.image_to_string(segmented_image, config='--psm 6')
    return recognized_text

# Function to process the recognized text using NLP techniques (Feature Engineering)
def process_text(recognized_text):
    # Tokenization using spaCy (Feature Engineering)
    doc = nlp(recognized_text)

    # Sentiment analysis using NLTK (Feature Engineering)
    sentiment_scores = sia.polarity_scores(recognized_text)
    sentiment = "Positive" if sentiment_scores["compound"] > 0 else "Negative" if sentiment_scores["compound"] < 0 else "Neutral"

    return recognized_text, sentiment

# Student's image path
student_image_path = list(uploaded_student.keys())[0]

# Teacher's image path
teacher_image_path = list(uploaded_teacher.keys())[0]

# Calling OCR function for the student and teacher images
student_recognized_text = perform_ocr(student_image_path)
teacher_recognized_text = perform_ocr(teacher_image_path)

# Processing the recognized text for the student and teacher
student_text, student_sentiment = process_text(student_recognized_text)
teacher_text, teacher_sentiment = process_text(teacher_recognized_text)

# Calculate semantic similarity between student and teacher text
student_doc = nlp(student_text)
teacher_doc = nlp(teacher_text)
similarity = student_doc.similarity(teacher_doc)

# Print results
print("Student Recognized Text:", student_text)
print("Student Sentiment:", student_sentiment)
print("\nTeacher Recognized Text:", teacher_text)
print("Teacher Sentiment:", teacher_sentiment)
print("\nSemantic Similarity Score:", similarity)

Saving s.jpeg to s (9).jpeg


Saving t.jpeg to t (9).jpeg


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Student Recognized Text: a ——————
PoOeUE
[EALse
mTeue
—

Student Sentiment: Neutral

Teacher Recognized Text: —

Teacher Sentiment: Neutral

Semantic Similarity Score: 0.7417896367704003
